# Exploring the Oura API

In [ ]:
import logging
import os
import pprint
from datetime import date, datetime, timedelta

import bokeh
import numpy as np
import pandas as pd
import panel as pn
from qself.oura import OuraAPIClient
from bokeh.io import output_notebook
from bokeh.models import ColumnDataSource, CrosshairTool, HoverTool
from bokeh.palettes import Category10
from bokeh.plotting import figure, show
from dotenv import load_dotenv

In [ ]:
logging.basicConfig(level=logging.INFO)

In [ ]:
pn.extension()
output_notebook()
load_dotenv()

In [ ]:
client = OuraAPIClient(os.environ["OURA_PERSONAL_ACCESS_TOKEN"])

## v2 API

### Daily Activity

In [ ]:
da = client("daily_activity", "2010-01-01", "2022-06-30")

In [ ]:
pprint.pprint(da["data"][0])

In [ ]:
sorted(da["data"][0]["contributors"].keys())

In [ ]:
days = []
act_calories = []
for d in da["data"]:
    days.append(d["day"])
    act_calories.append(d["active_calories"])

activity_df = pd.DataFrame(
    {"Date": pd.to_datetime(days), "Active_calories": act_calories}
)
assert len(activity_df) == len(activity_df["Date"].unique())
activity_df

In [ ]:
sorted(d.keys())

### Heart Rate

In [ ]:
e = datetime.now()
s = e - timedelta(hours=12)

In [ ]:
hr = client("heartrate", s.isoformat(), e.isoformat())
hr

### Personal Info

In [ ]:
pi = client("personal_info")
pi

### Sessions

In [ ]:
sessions = client("session", "2022-06-01", "2022-06-26")
sessions

### Tags

In [ ]:
tags = client("tag", "2010-01-01", "2022-06-30")
tags

In [ ]:
tag_df = pd.DataFrame(tags["data"])
tag_df["Date"] = pd.to_datetime(tag_df.day)
tag_df.drop(columns=["day"], inplace=True)
tag_df.rename(
    columns={"tags": "Tag", "text": "Text", "timestamp": "Timestamp"}, inplace=True
)
tag_df = tag_df[["Date", "Timestamp", "Tag", "Text"]]
tag_df = tag_df.explode("Tag")
tag_df["Date_following"] = tag_df["Date"] + timedelta(days=1)
tag_df

In [ ]:
tag_df.value_counts("Tag")

In [ ]:
tag_df.loc[tag_df["Tag"] == "tag_sleep_alcohol", :]

### Workouts 

In [ ]:
workouts = client("workout", "2022-06-01", "2022-06-26")
workouts

## v1 API

### Sleep

In [ ]:
sleep = client("sleep", "2010-01-01", "2022-06-30")
sleep

In [ ]:
dates = []
hr_average = []
hr_lowest = []
hrv = []
for day in sleep["sleep"]:
    try:
        d = day["summary_date"]
        hr_a = day["hr_average"]
        hr_l = day["hr_lowest"]
        rmssd = day["rmssd"]
    except KeyError:
        logging.info(f"Skipping {d}.")
        continue
    dates.append(d)
    hr_average.append(hr_a)
    hr_lowest.append(hr_l)
    hrv.append(rmssd)

hr_df = pd.DataFrame(
    {
        "Date": pd.to_datetime(dates),
        "HR_average": hr_average,
        "HR_lowest": hr_lowest,
        "HRV": hrv,
    }
)
hr_df["Year"] = hr_df["Date"].dt.isocalendar().year
hr_df["Month"] = hr_df["Date"].dt.month
hr_df["Week"] = hr_df["Date"].dt.isocalendar().week
hr_df

In [ ]:
hr_df.corr(method="pearson")

In [ ]:
hr_cds = ColumnDataSource(hr_df)
p = figure(title="Heart Rate", x_axis_type="datetime")
p.line(
    x="Date",
    y="HR_average",
    legend_label="HR average",
    line_color="gold",
    line_width=2,
    source=hr_cds,
)
p.line(
    x="Date",
    y="HR_lowest",
    legend_label="HR lowest",
    line_color="indigo",
    line_width=2,
    source=hr_cds,
)
p.legend.location = "top_right"
p.add_tools(CrosshairTool())
tooltips = [
    ("Date", "@Date{%F}"),
    ("HR average", "@{HR_average}"),
    ("HR lowest", "@{HR_lowest}"),
]
p.add_tools(HoverTool(tooltips=tooltips, formatters={"@Date": "datetime"}))
p.legend.location = "top_left"
p.legend.click_policy = "hide"
show(p)

### Monthly and weekly averages, compared year-on-year

In [ ]:
hr_year_df = (
    hr_df.groupby(["Year"]).agg(np.mean).reset_index().drop(["Week", "Month"], axis=1)
)

hr_month_df = (
    hr_df.groupby(["Year", "Month"]).agg(np.mean).reset_index().drop(["Week"], axis=1)
)
hr_month_df["Date"] = [f"{y}-{m}" for y, m in zip(hr_month_df.Year, hr_month_df.Month)]

hr_week_df = (
    hr_df.groupby(["Year", "Week"]).agg(np.mean).reset_index().drop(["Month"], axis=1)
)
hr_week_df["Date"] = [f"{y}-{w}" for y, w in zip(hr_week_df.Year, hr_week_df.Week)]

In [ ]:
hr_year_df

In [ ]:
def plot_year_on_year(x_axis_label: str, plot_hrv=False) -> bokeh.plotting.Figure:
    if x_axis_label == "Week":
        input_df = hr_week_df
    elif x_axis_label == "Month":
        input_df = hr_month_df
    else:
        raise ValueError(f"Unknown x_axis_label: {x_axis_label}")
    years = sorted(input_df.Year.unique())
    p = figure(title=f"Heart Rate ({x_axis_label})")
    for i, y in enumerate(years):
        hr_month_cds = ColumnDataSource(input_df.loc[input_df.Year == y, :])
        p.line(
            x=x_axis_label,
            y="HR_average",
            legend_label=f"HR average ({y})",
            line_color=Category10[10][i],
            line_width=2,
            source=hr_month_cds,
        )
        p.line(
            x=x_axis_label,
            y="HR_lowest",
            legend_label=f"HR lowest ({y})",
            line_color=Category10[10][i],
            line_width=2,
            source=hr_month_cds,
        )
        if plot_hrv:
            p.line(
                x=x_axis_label,
                y="HRV",
                legend_label=f"HRV ({y})",
                line_color=Category10[10][i],
                line_width=2,
                source=hr_month_cds,
            )
    p.legend.location = "top_right"
    p.add_tools(CrosshairTool())
    tooltips = [
        ("Date", "@{Date}"),
        ("HR average", "@{HR_average}"),
        ("HR lowest", "@{HR_lowest}"),
    ]
    if plot_hrv:
        tooltips.append(("HRV", "@{HRV}"))
    p.add_tools(HoverTool(tooltips=tooltips))
    p.legend.location = "top_right"
    p.legend.click_policy = "hide"
    return p

In [ ]:
kw = dict(x_axis_label=["Week", "Month"], plot_hrv=[False, True])
pn.interact(plot_year_on_year, **kw)

### Effect of previous day activity on HRV

In [ ]:
hr_activity_df = pd.merge(hr_df, activity_df, on="Date", how="inner")
hr_activity_df["Active_calories_shifted"] = hr_activity_df["Active_calories"].shift(1)
hr_activity_df

In [ ]:
# less than 300 active calories per day is considered implausible
hr_activity_df = hr_activity_df.loc[hr_activity_df["Active_calories_shifted"] > 300, :]

In [ ]:
hra_cds = ColumnDataSource(hr_activity_df)
p = figure(title="Effect of previous day activity on HRV")
p.circle(
    x="Active_calories_shifted",
    y="HRV",
    line_color="gold",
    legend_label="HRV",
    source=hra_cds,
)
p.legend.location = "top_right"
p.add_tools(CrosshairTool())
tooltips = [
    ("Date", "@Date{%F}"),
    ("Active calories", "@{Active_calories}"),
    ("HRV", "@{HRV}"),
]
p.add_tools(HoverTool(tooltips=tooltips, formatters={"@Date": "datetime"}))
p.legend.location = "top_left"
p.legend.click_policy = "hide"
show(p)

In [ ]:
hr_activity_df[["HRV", "Active_calories", "Active_calories_shifted"]].corr(
    method="pearson"
)

### Effect of previous day tag on HRV

In [ ]:
# join tag_df with next day's HR and HRV
tag_hr_df = pd.merge(
    hr_df, tag_df, left_on="Date", right_on="Date_following", how="inner"
)
if len(tag_hr_df) != len(tag_df):
    print(f"Only {len(tag_hr_df)} of {len(tag_df)} tags have HR data.")
tag_hr_df

In [ ]:
n = 100
hr_sample_df = hr_df.sample(n=n, random_state=0)
hr_sample_df

In [ ]:
thr_cds = ColumnDataSource(tag_hr_df)
p = figure(
    title="Effect of previous day tag on HRV",
    y_range=list(tag_hr_df.value_counts("Tag")[:10].index),
)
p.circle(
    x="HRV",
    y="Tag",
    line_color="blue",
    legend_label="HRV",
    source=thr_cds,
)
p.legend.location = "top_right"
p.add_tools(CrosshairTool())
tooltips = [
    ("Date", "@Date_x{%F}"),
    ("HRV", "@{HRV}"),
]
p.add_tools(HoverTool(tooltips=tooltips, formatters={"@Date_x": "datetime"}))
p.legend.location = "top_left"
p.legend.click_policy = "hide"
show(p)

### Activity Summaries

In [ ]:
activity = client("activity", "2022-06-01", "2022-06-26")
activity

### Readiness Summaries

In [ ]:
readiness = client("readiness", "2022-06-01", "2022-06-26")
readiness

### Ideal bedtime

In [ ]:
bedtime = client("bedtime", "2022-06-01", "2022-06-26")
bedtime